In [ ]:
# Catalog (=pg-stac) url from inside the docker network
stac_url="rs-server-catalog:8000"

import json
import requests
import urllib.request

# Clean old items and collections, if any
requests.delete(f"http://{stac_url}/collections/sentinel1-grd")

# Create a temp directory
import tempfile
tmp_dir = tempfile.TemporaryDirectory()
tmp_dirname = tmp_dir.name

In [ ]:
# TC 1: insert collection
urllib.request.urlretrieve(
    "https://raw.githubusercontent.com/stactools-packages/sentinel1/main/examples/grd/collection.json", 
    f"{tmp_dirname}/collection.json")

response = requests.post(f"http://{stac_url}/collections", data=open(f"{tmp_dirname}/collection.json", 'rb'))
response.raise_for_status()
json.loads(response.content)

In [ ]:
# TC2: Insert items into collection

urllib.request.urlretrieve(
    "https://raw.githubusercontent.com/stactools-packages/sentinel1/main/examples/grd/S1A_EW_GRDM_1SDH_20221130T014342_20221130T014446_046117_058549/S1A_EW_GRDM_1SDH_20221130T014342_20221130T014446_046117_058549.json", 
    f"{tmp_dirname}/item1.json")
urllib.request.urlretrieve(
    "https://raw.githubusercontent.com/stactools-packages/sentinel1/main/examples/grd/S1A_IW_GRDH_1SDV_20210809T173953_20210809T174018_039156_049F13/S1A_IW_GRDH_1SDV_20210809T173953_20210809T174018_039156_049F13.json", 
    f"{tmp_dirname}/item2.json")

response = requests.post(f"http://{stac_url}/collections/sentinel1-grd/items", data=open(f"{tmp_dirname}/item1.json", 'rb'))
response.raise_for_status()
json.loads(response.content)

response = requests.post(f"http://{stac_url}/collections/sentinel1-grd/items", data=open(f"{tmp_dirname}/item2.json", 'rb'))
response.raise_for_status()
json.loads(response.content)

In [ ]:
# Clean temp directory
tmp_dir.cleanup()

---
**NOTE**

STAC Swagger UI: http://localhost:8003/api.html

STAC browser: http://localhost:8081 (then load the STAC Catalog or API: `http://127.0.0.1:8003/catalog/sentinel1-grd`)

---